In [1]:
import os
import pandas as pd

from gensim.corpora import WikiCorpus
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import pandas as pd
import multiprocessing
import numpy as np

L = 10

path = '../data/new'
save_path = path + '/w2v'
if not os.path.exists(save_path):
    print(save_path)
    os.makedirs(save_path)


train = pd.read_csv(path + '/transaction_TRAIN_new.csv')
test = pd.read_csv(path + '/test_transaction_round2.csv')


# train = pd.read_csv(path + '/operation_TRAIN_new.csv')
# test = pd.read_csv(path + '/operation_round1_new.csv')

E:\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
E:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
print(train.head(5))

     UID  channel  day      time  trans_amt          amt_src1  \
0  19092      102   30  11:48:47       1459  acdbdb842ac20f1e   
1  13465      140   23  16:58:03        725  4d7831c6f695ab19   
2  13713      140   22  09:56:39        127  c5fc631370cabc0d   
3  22703      140    1  18:11:45      99317  a571c7fda8b7df37   
4  17816      140   22  13:04:02       3361  c5fc631370cabc0d   

           merchant code1 code2       trans_type1        ...         \
0  abc6bc660561e0eb   NaN   NaN  26bcf43a19df14c8        ...          
1  7a87b1aa20972d48   NaN   NaN  c2f2023d279665b2        ...          
2  cfd035fc7e0a53b2   NaN   NaN  c2f2023d279665b2        ...          
3  c71c876b8979028e   NaN   NaN  c2f2023d279665b2        ...          
4  6628f5ca013e3582   NaN   NaN  c2f2023d279665b2        ...          

                ip1   bal          amt_src2 acc_id2 acc_id3 geo_code  \
0  a664530c1fb20abe  1007  9fefed0a981dcb7a     NaN     NaN     wsbq   
1  52abe1e300a8aa10   100             

In [3]:
fea_name="merchant"
data = pd.concat([train, test]).reset_index(drop=True).sample(frac=1, random_state=2018)[["UID",fea_name]].fillna("missing")
tmp = data.groupby(["UID"])[fea_name].apply(lambda x:x.tolist()).reset_index()
print(tmp.head(5))
print(tmp.shape)
sentence =tmp[fea_name].tolist()
print(sentence[:5])
print('training...')


     UID                                           merchant
0  10000               [88aa547576f43f85, 88aa547576f43f85]
1  10001  [d1136ca5db98c376, f3237791ad270d6b, 3f6d3d0f4...
2  10002  [5b9283139b4920cc, 8f57527418b3f457, 5b9283139...
3  10003  [3bd5cf7c40962299, 3bd5cf7c40962299, be2366a74...
4  10004  [d67f4667ac10372d, d67f4667ac10372d, 3bd5cf7c4...
(61135, 2)
[['88aa547576f43f85', '88aa547576f43f85'], ['d1136ca5db98c376', 'f3237791ad270d6b', '3f6d3d0f42519ea4', 'a18cb3d84b9ec04c', 'd8babe2d19fa0c08', 'fd1e5bf59a1ee156', 'a18cb3d84b9ec04c', '3bd5cf7c40962299', 'a18cb3d84b9ec04c', 'a18cb3d84b9ec04c', 'a18cb3d84b9ec04c', 'd1136ca5db98c376', 'a18cb3d84b9ec04c', '3f6d3d0f42519ea4', 'f3237791ad270d6b', 'd8babe2d19fa0c08', 'fd1e5bf59a1ee156', 'd8babe2d19fa0c08'], ['5b9283139b4920cc', '8f57527418b3f457', '5b9283139b4920cc'], ['3bd5cf7c40962299', '3bd5cf7c40962299', 'be2366a746e08a07', '3bd5cf7c40962299'], ['d67f4667ac10372d', 'd67f4667ac10372d', '3bd5cf7c40962299', 'd67f4667ac10372d',

In [13]:
model = Word2Vec(sentence, size=L, window=2, min_count=1, workers=multiprocessing.cpu_count(),
                 iter=10)
print('outputing...')



for fea in [fea_name]:
    values = []
    for line in list(data[fea].values):
        values.append(line)
    values = set(values)
    print(len(values))
    w2v = []
    for i in values:
        a = [i]
        a.extend(model[str(i)])
        w2v.append(a)
    out_df = pd.DataFrame(w2v)

    name = [fea]
    for i in range(L):
        name.append(name[0] + 'W' + str(i))
    out_df.columns = name
    out_df.to_csv(save_path + '/' + fea + 'new_.csv', index=False)

outputing...
34971


E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()
